### Importing camera.csv data into Camera

In [3]:
import pymongo
from pymongo import MongoClient
import csv

# ── Your starter connection ──
client = MongoClient("172.22.32.1", 27017) 
db = client.fit3182_db
camera_coll = db.Camera

# ── Skip if already imported ──
if camera_coll.estimated_document_count() > 0:
    print("Camera collection already contains data. Skipping import.")
else:
    idx_name = camera_coll.create_index(
        [("pos", pymongo.ASCENDING)],
        name="pos_idx"
    )
    print(f"Ensured index on 'pos': {idx_name}")
    
    # ── Load CSV and insert ──
    csv_path = 'data/camera.csv'
    docs = []
    with open(csv_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            docs.append({
                "_id":        int(row['camera_id']),
                "lat":        float(row['latitude']),
                "long":       float(row['longitude']),
                "pos":        float(row['position']),
                "speed_limit": int(row['speed_limit'])
            })

    if docs:
        result = camera_coll.insert_many(docs)
        print(f"Inserted {len(result.inserted_ids)} camera documents.")
        print("Current indexes on Camera:")
        for name, info in camera_coll.index_information().items():
            print(f" • {name}: {info['key']}")
    else:
        print("No camera records found in CSV.")


Ensured index on 'pos': pos_idx
Inserted 3 camera documents.
Current indexes on Camera:
 • _id_: [('_id', 1)]
 • pos_idx: [('pos', 1)]


### Importing vehicle.csv data into Vehicle

In [4]:
import csv
from datetime import datetime
from pymongo import MongoClient

# ── Connection ──
client = MongoClient("172.22.32.1", 27017)
db = client.fit3182_db
vehicle_coll = db.Vehicle

if vehicle_coll.estimated_document_count() > 0:
    print("Vehicle collection already contains data. Skipping import.")
else:
    # ── Prepare sets & counters ──
    existing_ids = set(vehicle_coll.distinct('_id'))
    seen_in_file = set()
    docs_to_insert = []
    update_count = 0
    added_count = 0

    csv_path = 'data/vehicle.csv'
    with open(csv_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            plate = row['car_plate']

            # Parse the incoming registration_date
            ts = row['registration_date'].rstrip("Z")
            reg_date = datetime.fromisoformat(ts)

            if plate in seen_in_file:
    #             print("Found duplicate:")
                # Plate already in DB → check whether to update
                existing = vehicle_coll.find_one(
                    {"_id": plate}
                )
    #             print(existing)
                if existing and reg_date > existing['registration_date']:
    #                 print("Registration is newer, updating...")
                    # Only update if the CSV date is newer
                    vehicle_coll.update_one(
                        {"_id": plate},
                        {"$set": {
                            "registration_date": reg_date,
                            "owner_name":        row['owner_name'],
                            "owner_addr":        row['owner_addr'],
                            "vehicle_type":      row['vehicle_type']
                        }}
                    )
                    update_count += 1
            else:
                seen_in_file.add(plate)
                # Brand-new plate → schedule for insert
                vehicle_coll.insert_one({
                    "_id":               plate,
                    "owner_name":        row['owner_name'],
                    "owner_addr":        row['owner_addr'],
                    "vehicle_type":      row['vehicle_type'],
                    "registration_date": reg_date
                })
                added_count += 1

    # ── Do the batch insert, if any ──
    if added_count > 0:
        print(f"Inserted {added_count} new vehicle documents.")
    else:
        print("No new vehicle records to insert.")

    # ── Report on any updates we made ──
    if update_count:
        print(f"Updated {update_count} existing vehicle document{'s' if update_count>1 else ''}.")

    # ── (Optional) show your indexes ──
    print("Current indexes on Vehicle:")
    for name, info in vehicle_coll.index_information().items():
        print(f" • {name}: {info['key']}")


Inserted 9844 new vehicle documents.
Updated 69 existing vehicle documents.
Current indexes on Vehicle:
 • _id_: [('_id', 1)]


### Data Design and Streaming using Apache Structured Streaming

In [15]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.3.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,org.mongodb.spark:mongo-spark-connector_2.12:10.1.1 pyspark-shell'
from pymongo import MongoClient
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, element_at, when
from pyspark.sql.streaming import StreamingQueryException

from pyspark.sql.types import (
    StructType, StringType, IntegerType, DoubleType, TimestampType
)
from pyspark.sql.functions import (
    col, expr, from_json
)
import uuid

In [6]:
topic_name = 'camera_events'
hostip = "172.22.32.1" # change it to your IP

In [7]:
spark = (
    SparkSession.builder
    .master('local[*]')
    .appName('Apache Spark Structured Streaming from Kafka for Violation Detection')
    .getOrCreate()
)

In [8]:
eventSchema = StructType() \
    .add("batch_id", IntegerType()) \
    .add("event_id", StringType()) \
    .add("car_plate", StringType()) \
    .add("camera_id", IntegerType()) \
    .add("timestamp", TimestampType()) \
    .add("speed_reading", DoubleType()) \
    .add("producer", StringType()) \
    .add("sent_at", TimestampType())

def read_camera_stream(suffix: str):
    return (
        spark.readStream
             .format("kafka")
             .option("kafka.bootstrap.servers", f"{hostip}:9092")
             .option("subscribe", f"{topic_name}_{suffix}")
             .load()
             .selectExpr("CAST(value AS STRING) as json")
             .select(from_json(col("json"), eventSchema).alias("data"))
             .select("data.*")
             # watermark on the event time to purge old state
             .withWatermark("sent_at", "10 minute")
    )

In [9]:
streamA = read_camera_stream("a")
streamB = read_camera_stream("b")
streamC = read_camera_stream("c")

A_vm = streamA.alias("A")
B_vm = streamB.alias("B")
C_vm = streamC.alias("C")

In [10]:
AB = A_vm.join(
    B_vm,
    on=[
        col("A.car_plate")==col("B.car_plate")
      ],
    how="inner"
).filter(
    (col("B.timestamp")  > col("A.timestamp"))
).select(
    # you can pick whichever columns you need downstream
    col("A.*"), 
    col("B.camera_id").alias("camera_id_B"),
    col("B.car_plate").alias("car_plate_B"),
    col("B.timestamp").alias("timestamp_B"),
    col("B.speed_reading").alias("speed_reading_B")
).alias("AB")

ABC = AB.join(
    C_vm,
    on=[
        col("AB.car_plate")==col("C.car_plate"),
        col("C.sent_at") >= col("AB.sent_at") - expr("INTERVAL 300 SECONDS"),
        col("C.sent_at") <= col("AB.sent_at") + expr("INTERVAL 300 SECONDS")
      ],
    how="inner"
).filter(
    (col("C.timestamp")  > col("AB.timestamp_B"))
).select(
    col("AB.*"), 
    col("C.camera_id").alias("camera_id_C"),
    col("C.car_plate").alias("car_plate_C"),
    col("C.timestamp").alias("timestamp_C"),
    col("C.speed_reading").alias("speed_reading_C")
).alias("ABC")

In [11]:
# for debugging, print on console
class ConsoleWriter:
    def open(self, partition_id, epoch_id):
        self.mongo_client = MongoClient(
            host=f'{hostip}',
            port=27017
        )
        self.db = self.mongo_client['fit3182_db']
        self.camera_coll = self.db.Camera
        return True
    
    def process(self, row):
        data = row.asDict()
        print(f"\ncurrent data: {data}")
#         currentCameraData = self.camera_coll.find_one(
#                     {"_id": data["camera_id"]}
#                 )
#         print(f'current camera: {currentCameraData["_id"]}')
#         if float(currentCameraData["speed_limit"]) < float(data["speed_reading"]):
#             print(f"instant speed violated: {currentCameraData['speed_limit']} vs {data['speed_reading']}")
# #             self.db["Violation"].insert_one(self.createJsonSchema(data)) 
#             print(f"Generated schema: {self.createJsonSchema(data, currentCameraData['speed_limit'])}")
        
    # called once all rows have been processed (possibly with error)
    def close(self, err):
        self.mongo_client.close()
        
    def createJsonSchema(self, data, speed_limit):
        # parse timestamp into a datetime if needed
        ts = data["timestamp"]
        if isinstance(ts, str):
            ts = datetime.fromisoformat(ts)

        # compute the “date” bucket = midnight on that day
        date_bucket = datetime(ts.year, ts.month, ts.day)

        # build the single-infraction subdoc
        violation = {
            "type":             "instantaneous",
            "camera_id_start":  data["camera_id"],
            "camera_id_end":    None,
            "timestamp_start":  ts,
            "timestamp_end":    None,
            "measured_speed":   float(data["speed_reading"]),
            "speed_limit":      float(speed_limit)
        }

        return {
            "violation_id": str(uuid.uuid4()),  # or f"{data['car_plate']}_{date_bucket.date()}"
            "car_plate":    data["car_plate"],
            "date":         date_bucket,
            "violations":   [violation]
        }

In [13]:
class DbWriter:
    def open(self, partition_id, epoch_id):
        self.mongo_client = MongoClient(
            host=f'{hostip}',
            port=27017
        )
        self.db = self.mongo_client['fit3182_db']
        self.camera_coll = self.db.Camera
        self.violation_coll = self.db.Violation
        return True
    
    def process(self, row):
        data = row.asDict()
        currentCameraData = self.camera_coll.find_one(
                    {"_id": data["camera_id"]}
                )
        if float(currentCameraData["speed_limit"]) < float(data["speed_reading"]):
            self.addViolation(data, currentCameraData["speed_limit"])
        
    # called once all rows have been processed (possibly with error)
    def close(self, err):
        self.mongo_client.close()
        
    def addViolation(self, data, speed_limit):
        # parse timestamp into a datetime if needed
        ts = data["timestamp"]
        if isinstance(ts, str):
            ts = datetime.fromisoformat(ts)

        # compute the “date” bucket = midnight on that day
        date_bucket = datetime(ts.year, ts.month, ts.day)

        # build the single-infraction subdoc
        violation = {
            "type":             "instantaneous",
            "camera_id_start":  data["camera_id"],
            "camera_id_end":    None,
            "timestamp_start":  ts,
            "timestamp_end":    None,
            "measured_speed":   float(data["speed_reading"]),
            "speed_limit":      float(speed_limit)
        }
        
        existing = self.violation_coll.find_one(
                    {"car_plate": data["car_plate"], "date": date_bucket}
                )
        
        if existing:
            print(existing)
            originalViolations = existing["violations"]
            originalViolations.append(violation)
            self.violation_coll.update_one(
                        {"car_plate": data["car_plate"], "date": date_bucket},
                        {"$set": {
                            "violations": originalViolations,
                        }}
                    )
            print(f"added violation to carplate {data['car_plate']} at {date_bucket}")
        else:
            self.violation_coll.insert_one(
                    {
                    "violation_id": str(uuid.uuid4()),  # or f"{data['car_plate']}_{date_bucket.date()}"
                    "car_plate":    data["car_plate"],
                    "date":         date_bucket,
                    "violations":   [violation]
                }
            )

In [14]:
writer_stream = (
    ABC.writeStream.format("Console")
    .option("checkpointLocation", "./stream_checkpoints_3")
    .outputMode('append').foreach(ConsoleWriter())
)
writer_stream = (
    ABC.writeStream.format("Console")
    .option("checkpointLocation", "./streamA_checkpoints")
    .outputMode('append').foreach(DbWriter())
)

In [ ]:
try:
    query = writer_stream.start()
    query.awaitTermination()
except KeyboardInterrupt:
    print('Interrupted by CTRL-C. Stopped query')
except StreamingQueryException as exc:
    print(exc)
finally:
    query.stop()